In [1]:
import numpy as np
import torch
from NeuralNet import NeuralNetwork as Network
import time
from Utilities import Utilities as Utils
from IPython.display import clear_output

In [5]:
Filters = 128
Layers = 15
HistoryDepth = 8
KernalSize = 3

model = Network(Filters, HistoryDepth + 1, Layers, kernal_size=KernalSize)

model.load_state_dict(torch.load('../../Models/HumanModels/128_15.pt', map_location=torch.device('cpu')))
model.eval()

NeuralNetwork(
  (conv_layer): ConvolutionLayer(
    (conv2d_sequential): Sequential(
      (0): Conv2d(9, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (residual_layers): ModuleList(
    (0-14): 15 x ResidualLayer(
      (conv2d_sequential): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
  )
  (policy_head): PolicyHead(
    (head): Sequential(
      (0): Conv2d(128, 2, kernel_size=(1, 1), stride=(1, 1))
      (1): Flatten(start_dim=1, end_dim=-1)
      (2): BatchNorm1d(450, eps=1e-05, momentu

In [18]:
board = np.zeros((HistoryDepth + 1, 15, 15), dtype=bool)

In [19]:
player = 1
while (True):
    if player % 2:
        modelInput = torch.from_numpy(board.astype(np.float32)).unsqueeze(0)
        with torch.no_grad():
            output = model(modelInput)
        policyOut, valueOut = output
        policyOut = torch.nn.functional.softmax(policyOut, dim=1)
        policyOut = np.array(policyOut)
        policyOut = policyOut.reshape(15, 15)
        policyOut *= ~board[HistoryDepth]
        policyOut *= ~board[HistoryDepth // 2]
        policyOut = policyOut.flatten()

        valueOut = valueOut[0].item()

        index = policyOut.argmax()
        x, y = int(index // 15), int(index % 15)
        board = Utils.makeMove(board, x, y)
        clear_output(wait=True)
        print(x,y)
        print(Utils.sliceGamestate(board, 0))
        time.sleep(1)
    else:
        x = int(input("X:"))
        y = int(input("Y:"))

        board = Utils.makeMove(board, x, y)
        clear_output(wait=True)
        print(Utils.sliceGamestate(board, 0))
        time.sleep(1)
    player += 1

7 8
   --------------------------------------------------------------
14 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
13 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
12 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
11 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
10 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 9 |   |   |   |   |   |   |   |   |   |   |   |   |   |   |   |
   --------------------------------------------------------------
 8 |   |   |   |   |   |   |   | W | B |   |   |   |   |   |   |
   --------------------------------------------------------------
 7 |   |   | 

ValueError: invalid literal for int() with base 10: ''